In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Redback_A/toxic.csv")
df = df.drop("id", axis=1)
df.columns = ['comment', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df.head()

,comment,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
## The data is very imbalanced
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
toxic_comments = df[df[classes].sum(axis=1)>0]
clean_comments = df[df[classes].sum(axis=1)==0]

print("Number of toxic comments", len(toxic_comments))
print("Number of clean comments", len(clean_comments))

Number of toxic comments 16225
Number of clean comments 143346


In [5]:
## Sample clean comments and make dataset balanced
uniform_dataset = pd.concat([toxic_comments, clean_comments.sample(len(toxic_comments))])

In [6]:
## The data is now balanced
toxic_comments = uniform_dataset[uniform_dataset[classes].sum(axis=1)>0]
clean_comments = uniform_dataset[uniform_dataset[classes].sum(axis=1)==0]

print("Number of toxic comments", len(toxic_comments))
print("Number of clean comments", len(clean_comments))

Number of toxic comments 16225
Number of clean comments 16225


In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.8 MB/s eta 0:00:00


In [19]:
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(uniform_dataset["comment"], uniform_dataset.drop("comment", axis=1), test_size=0.3, random_state=42)

In [20]:
# Tokenize the text data using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
X_test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

In [21]:
# Convert the encoded features and labels into PyTorch tensors
X_train_input_ids = torch.tensor(X_train_encodings['input_ids'])
X_train_attention_mask = torch.tensor(X_train_encodings['attention_mask'])
y_train = torch.tensor(y_train.values, dtype=torch.float32)

X_test_input_ids = torch.tensor(X_test_encodings['input_ids'])
X_test_attention_mask = torch.tensor(X_test_encodings['attention_mask'])
y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [11]:
# Create a PyTorch DataLoader object for training and testing data
train_dataset = TensorDataset(X_train_input_ids, X_train_attention_mask, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = TensorDataset(X_test_input_ids, X_test_attention_mask, y_test)
test_loader = DataLoader(test_dataset, batch_size=16)

In [12]:
# Create a BERT-based multilabel classification model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Count the number of batches in train_loader
import math

batch_size = 16  # set the batch size

num_batches = math.ceil(len(train_dataset) / batch_size)
print(f"Number of batches in train_loader: {num_batches}")

Number of batches in train_loader: 1420


In [ ]:
import random
import torch
import torch.nn as nn

# Train the model using a suitable optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
num_epochs = 1
batch_size = 16
train_subset_size = 200  # number of batches to use for training
random_seed = 42  # set a random seed for reproducibility

model.train()

criterion = nn.BCEWithLogitsLoss()

# set the random seed
random.seed(random_seed)

# convert train_loader to a list of batches
train_batches = list(train_loader)

for epoch in range(num_epochs):
    # randomly select a subset of batches for training
    train_subset = random.sample(train_batches, train_subset_size)
    
    for batch in train_subset:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        # get the model outputs
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # get the logits and labels tensors
        logits = outputs.logits
        labels = labels.float()

        # compute the loss
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        # print the average loss for the current batch
        print(f"Batch loss: {loss.mean().item()}")

Batch loss: 0.2802700400352478
Batch loss: 0.3621496558189392
Batch loss: 0.3402864336967468
Batch loss: 0.26903846859931946
Batch loss: 0.2913309931755066
Batch loss: 0.369019091129303
Batch loss: 0.35422438383102417
Batch loss: 0.3161960244178772
Batch loss: 0.29932448267936707
Batch loss: 0.41158199310302734
Batch loss: 0.3162040412425995
Batch loss: 0.25558599829673767
Batch loss: 0.45366621017456055
Batch loss: 0.31916430592536926
Batch loss: 0.3264186382293701
Batch loss: 0.178269624710083
Batch loss: 0.26376351714134216
Batch loss: 0.3049768805503845
Batch loss: 0.3110998272895813
Batch loss: 0.1802365481853485
Batch loss: 0.2746927738189697
Batch loss: 0.26433566212654114
Batch loss: 0.2976924180984497
Batch loss: 0.2231234312057495
Batch loss: 0.26237452030181885
Batch loss: 0.20801937580108643
Batch loss: 0.19453683495521545
Batch loss: 0.28710731863975525
Batch loss: 0.32890570163726807
Batch loss: 0.26715904474258423
Batch loss: 0.26265597343444824
Batch loss: 0.21231877803

In [ ]:
# Set the model to evaluation mode
model.eval()

# Create empty lists for storing true labels and predicted labels
true_labels = []
pred_labels = []

# Iterate through the test data and make predictions
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)

        # Convert the logits to probabilities and round off to 0 or 1
        probs = torch.sigmoid(outputs.logits)
        preds = (probs >= 0.5).float()

        true_labels.extend(labels.cpu().detach().numpy().tolist())
        pred_labels.extend(preds.cpu().detach().numpy().tolist())

In [ ]:
# Calculate evaluation metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='micro')
recall = recall_score(true_labels, pred_labels, average='micro')
f1 = f1_score(true_labels, pred_labels, average='micro')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 score:', f1)

Accuracy: 0.6707755521314843
Precision: 0.7722517730496454
Recall: 0.8273340298223952
F1 score: 0.7988445137328625
